### Install and import dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
      --------------------------------------- 0.0/2.5 MB 151.3 kB/s eta 0:00:17
      --------------------------------------- 0.0/2.5 MB 151.3 kB/s eta 0:00:17
      --------------------------------------- 0.0/2.5 MB 151.3 kB/s eta 0:00:17
      --------------------------------------- 0.0/2.5 MB 151.3 kB/s eta 0:00:17
     -- ------------------------------------- 0.1/2.5 MB 315.1 kB/s eta 0:00:08
     -- ------------------------------------- 0.1/2.5 MB 315.1 kB/s eta 0:00:08
     -- ------------------------------------- 0.1/2.5 MB 315.1 kB/s eta 0:00:08
     -- ------------------

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

### Encode and Calculate Sentiment

In [36]:
tokens = tokenizer.encode('I am feeling happy but not that much', return_tensors='pt')

In [37]:
tokens

tensor([[  101,   151, 10345, 35260, 19308, 10502, 10497, 10203, 12977,   102]])

In [38]:
result = model(tokens)

In [39]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7688, -0.0713,  1.6600,  1.0543, -0.8139]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [40]:
result.logits

tensor([[-1.7688, -0.0713,  1.6600,  1.0543, -0.8139]],
       grad_fn=<AddmmBackward0>)

In [41]:
# Sentiment rating 1-5
int(torch.argmax(result.logits))+1

3

### Collect reviews 

In [72]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')

In [73]:
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})

In [81]:
results[0]

<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Great coffee and vibe. That's all  you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. <br/><br/>This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent.</span></p>

In [77]:
reviews = [result.text for result in results]

In [82]:
reviews[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

### Load Reviews into DataFrame and Score

In [83]:
import numpy as np
import pandas as pd

In [84]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [85]:
df.head()

,review
0,Great coffee and vibe. That's all you need. C...
1,Great coffee and vibe. That's all you need. C...
2,Great food amazing coffee and tea. Short walk ...
3,It was ok. Had coffee with my friends. I'm new...
4,Ricotta hot cakes! These were so yummy. I ate ...


In [89]:
df.shape

(11, 1)

In [86]:
df['review'].iloc[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [87]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [88]:
sentiment_score(df['review'].iloc[1])

4

In [90]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [92]:
df.head()

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,Great food amazing coffee and tea. Short walk ...,5
3,It was ok. Had coffee with my friends. I'm new...,3
4,Ricotta hot cakes! These were so yummy. I ate ...,5


In [93]:
df['review'].iloc[3]

"It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places."